# load weather data

In [1]:
weather = pd.read_csv('../data/weather.csv', index_col = 0)

In [2]:
weather = weather.reset_index()

# 분류 내용

### 유형 1. moderate한 날짜 vs 그렇지 않은날짜  >> 카테고리 2개

### 유형 2. codesum조합에 대해 빈도수 별 분류 >> 카테고리 ??
     1. 100번 이상 발생한 것
     2. 200번 이상 발생한 것

## 유형 1. moderate한 날짜 vs 그렇지 않은 날짜 분류

In [3]:
weather_v1 = weather.copy()

In [4]:
def distribution_codesum(codesum):
    if codesum == " ":
        return "MO"
    else:
        return "Not_MO"

In [5]:
weather_v1['codesum'] = weather_v1['codesum'].apply(distribution_codesum)

In [6]:
weather_v1.head()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,Not_MO,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,Not_MO,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,MO,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,MO,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,MO,0.0,0.00,29.95,30.47,14.0,36,13.8


## codesum 이 Moderate 한지 아닌지 2가지로 분류한 weather를 weather_v1.csv 로 저장

In [7]:
weather_v1.to_csv('../data/weather_v1.csv', sep = ',')

## 유형 2. codesum조합에 대해 빈도수 별 분류 >> 카테고리 ??
     1. 100번 이상 발생한 것
     2. 200번 이상 발생한 것

# moderate한 날짜에 `MO` 채워넣기

In [8]:
weather_v2 = weather.copy()

In [9]:
def fill_value_codesum(codesum):
    if codesum == " ":
        return "MO"
    else:
        return codesum

In [10]:
data = pd.DataFrame(weather_v2, columns=['date', 'codesum'])

In [11]:
data['codesum'] = data['codesum'].apply(fill_value_codesum)

In [12]:
data.tail()

,date,codesum
20512,2014-10-31,MO
20513,2014-10-31,RA
20514,2014-10-31,MO
20515,2014-10-31,RA SN
20516,2014-10-31,MO


## 유형 2-1

# codesum 데이터를 읽기 좋게 바꾸기
- ex "FG+ EXEX EX" -> "FG,EX,EX,EX" 이런식으로


In [13]:
def ch_codesum(codesum):
    result = ""
    
    codesum = codesum.replace(" ","")
    codesum = codesum.replace("+","")
    
    for s in range(0,len(codesum)):
        if s % 2 == 0 and s != 0:
            result += ',' + codesum[s]
        else:
            result += codesum[s]
    return result

In [14]:
data['codesum'] = data['codesum'].apply(ch_codesum)

# Counter 함수를 이용해 codesum마다 얼만큼 쓰였는지 확인

In [15]:
from collections import Counter

In [16]:
tmp_list = list(data['codesum'])

In [17]:
tmp_list[:10]

['RA,FZ,FG,BR',
 'RA',
 'MO',
 'MO',
 'MO',
 'MO',
 'RA,BR',
 'UP',
 'FG,FG,BR',
 'FG,FG,BR']

#### + 가 있는 경우 2개 중복이 되므로 이부분은 set을 이용해 없애주자

In [18]:
tmp_list[8]

'FG,FG,BR'

In [19]:
list(set(tmp_list[8].split(',')))

['FG', 'BR']

In [20]:
counts = []
for i in tmp_list:
    i = list(set(i.split(',')))
    for j in i:
        counts.append(j)

In [21]:
counts[:10]

['BR', 'RA', 'FG', 'FZ', 'RA', 'MO', 'MO', 'MO', 'MO', 'RA']

In [22]:
counts_codesum = dict(Counter(counts))

# `codesum` : `갯수`       `퍼센트(%)`

In [23]:

for idx, val in counts_codesum.items():
    sum_val = 0
    sum_percentage = 0
    tmp_percentage = 0
    percentage = round((val / 20517) * 100,2)
    
    print(idx,":",val," "*(10 - len(str(val))),str(percentage)+"%")
    tmp_percentage += percentage
    

DU : 28          0.14%
SN : 1008        4.91%
FG : 1307        6.37%
FZ : 332         1.62%
SQ : 14          0.07%
BL : 59          0.29%
VC : 571         2.78%
FU : 33          0.16%
RA : 5325        25.95%
MI : 52          0.25%
PR : 1           0.0%
PL : 20          0.1%
MO : 11051       53.86%
HZ : 1482        7.22%
UP : 230         1.12%
GS : 5           0.02%
GR : 3           0.01%
SG : 1           0.0%
DZ : 497         2.42%
BR : 6359        30.99%
TS : 1888        9.2%
BC : 111         0.54%


In [24]:
counts_set = []
for i in tmp_list:
    i = ','.join(list(set(i.split(','))))
    counts_set.append(i)

In [25]:
counts_set[:10]

['BR,RA,FG,FZ', 'RA', 'MO', 'MO', 'MO', 'MO', 'RA,BR', 'UP', 'FG,BR', 'FG,BR']

In [26]:
counts_codesum_set = dict(Counter(counts_set))

# 모든 codesum 조합에 대해 묶어서 100개 이상만 뽑았을 때
- 전체의 90%를 차지
- 18개의 카테고리 + 1 = 0
- 같은 코드섬이 2개 들어가 있는 경우는 그 코드에 +가 있었던 경우임

In [27]:
code_name = []
sum_percentage = 0
for idx,val in counts_codesum_set.items():
    
    if val > 100:
        percentage = round((val / 20517) * 100,2)
        print(idx," "*(12 - len(idx)),val," "*(7 - len(str(val))),str(percentage)+"%")
        sum_percentage += percentage
        for i in idx.split(','):
            code_name.append(i)
            
print("-------total_percentage---------")
print(str(sum_percentage) + "%")

VC,RA,TS      132      0.64%
RA,BR         1476     7.19%
HZ            365      1.78%
FG,BR         354      1.73%
BR            1431     6.97%
SN            198      0.97%
RA,BR,TS      481      2.34%
SN,BR         244      1.19%
RA,BR,DZ      126      0.61%
BR,HZ         400      1.95%
MO            11051    53.86%
TS            189      0.92%
RA            1281     6.24%
BR,RA,FG      251      1.22%
RA,TS         416      2.03%
RA,BR,HZ      147      0.72%
VC,RA,BR,TS   207      1.01%
-------total_percentage---------
91.36999999999999%


In [28]:
set(code_name)

{'BR', 'DZ', 'FG', 'HZ', 'MO', 'RA', 'SN', 'TS', 'VC'}

### 카테고리를 좀 더 줄이고자 고민하면서 현재 위의 경우 `RA BR TS VC`, `BR RA TS` 가 다르게 인식이 되고 있다. 그래서 그냥 위의 경우 `RA BR TS` 로 분류하고 싶어졌다

### 2개까지 잡냐 or 3개까지 잡냐... 둘다 해보자

### 2개부터

# codesum이 2개 이상 기록된 데이터를 분류하자

In [29]:
counts_set = []
for i in tmp_list:
    i = ','.join(list(set(i.split(','))))
    counts_set.append(i)

In [30]:
counts_set[:10]

['BR,RA,FG,FZ', 'RA', 'MO', 'MO', 'MO', 'MO', 'RA,BR', 'UP', 'FG,BR', 'FG,BR']

In [31]:
exp_list = []
for i in counts_set:
    if len(i.split(',')) == 2:
        exp_list.append(i)

In [32]:
exp_dict = dict((Counter(exp_list)))

In [33]:
exp = list(exp_dict)

In [34]:
sum_count = 0
for idx in exp:
    count = 0
    # 2개씩 조합해서 비교
    for code in counts_set:
        code = code.split(',')
        if len(code) >= 2:
            for i in range(0,len(code)-1):
                if idx.split(',') == list(set(code))[i:i+2]:
                    count += 1
    sum_count += count
    
    print(idx,":",count)
print('총합',sum_count)

DU,BR : 4
SN,RA : 225
FU,HZ : 5
RA,DZ : 34
FU,TS : 1
RA,BR : 3292
UP,RA : 47
FG,FZ : 24
BR,TS : 990
PL,SN : 6
UP,SN : 114
HZ,TS : 10
RA,TS : 576
DU,RA : 1
SQ,TS : 1
DZ,BR : 143
FG,MI : 24
VC,FG : 30
SN,HZ : 36
FG,BC : 16
FU,BR : 10
DZ,FG : 2
SN,BR : 341
RA,HZ : 80
BR,HZ : 776
PL,RA : 9
VC,TS : 78
SN,BL : 14
DU,HZ : 6
FG,BR : 665
RA,FZ : 9
DU,BL : 8
DZ,FZ : 10
SQ,BR : 2
DZ,HZ : 37
RA,FG : 15
SN,DZ : 24
UP,HZ : 4
UP,BR : 6
총합 7675


# 1개인것만

In [35]:
exp_list = []
for i in counts_set:
    if len(i.split(',')) == 1:
        exp_list.append(i)

In [36]:
exp_dict = dict(Counter(exp_list))

In [37]:
for idx,val in exp_dict.items():
    if val > 100:
        print(idx, val)

SN 198
MO 11051
HZ 365
TS 189
RA 1281
BR 1431


### 결과를 보니 20517보다 개수가 많다. 그이유는?? 예를들어 `RA BR SN` 인 경우 `RA BR`, `BR SN` 에 둘다 count를 했기 때문

# 3개를 기준으로 해보면?

In [38]:
exp_list = []
for i in counts_set:
    if len(i.split(',')) == 3:
        exp_list.append(i)

In [39]:
exp_dict = dict(Counter(exp_list))

In [40]:
exp = list(exp_dict)

In [41]:
sum_count = 0
for idx in exp:
    count = 0
    # 4개 중 3개씩 조합해서 비교
    for code in counts_set:
        code = code.split(',')
        if len(code) >= 3 and idx.split(',') == list(set(code))[:3]:

            count += 1
    sum_count += count
    
    print(idx,":",count)
print('총합',sum_count)

SN,DZ,BR : 16
UP,RA,FZ : 2
RA,FG,TS : 7
UP,BR,HZ : 3
HZ,BR,TS : 0
SN,FG,FZ : 17
VC,BR,TS : 20
DZ,BR,TS : 2
PL,RA,FZ : 2
SN,RA,DZ : 3
FG,BR,FZ : 34
PL,RA,BR : 5
RA,BR,TS : 559
SN,GS,TS : 1
BR,RA,FG : 0
DZ,FG,HZ : 1
SN,FG,BR : 99
SN,HZ,BL : 3
UP,SN,RA : 40
SN,BR,HZ : 62
VC,RA,TS : 151
UP,SN,BL : 4
RA,HZ,DZ : 11
RA,TS,DZ : 4
FG,FZ,BC : 1
SN,RA,BR : 114
DZ,BR,HZ : 23
FU,FG,MI : 0
FG,HZ,BC : 1
RA,TS,FZ : 1
FU,BR,TS : 2
DZ,FZ,RA : 1
DZ,BR,FZ : 7
BR,FG,MI : 3
SN,RA,FG : 1
FG,BR,HZ : 67
VC,TS,HZ : 4
BR,GS,HZ : 0
SQ,RA,TS : 1
SN,BR,TS : 1
FG,RA,MI : 3
FG,BR,BC : 54
SN,BR,BL : 5
UP,SN,BR : 28
RA,BR,FZ : 21
FG,BR,TS : 7
DU,HZ,BL : 1
BR,DZ,FG : 0
UP,SN,HZ : 5
VC,HZ,TS : 5
FU,BR,HZ : 3
RA,BR,HZ : 147
FU,RA,BR : 0
DU,BR,HZ : 3
PL,RA,HZ : 1
RA,BR,DZ : 150
UP,RA,DZ : 1
UP,RA,BR : 26
VC,FG,BR : 4
RA,HZ,TS : 0
RA,GS,TS : 2
총합 1739


# codesum 조합을 다시 만들자 조합이 3개인것 까지만!

In [42]:
counts_codesum

{'BC': 111,
 'BL': 59,
 'BR': 6359,
 'DU': 28,
 'DZ': 497,
 'FG': 1307,
 'FU': 33,
 'FZ': 332,
 'GR': 3,
 'GS': 5,
 'HZ': 1482,
 'MI': 52,
 'MO': 11051,
 'PL': 20,
 'PR': 1,
 'RA': 5325,
 'SG': 1,
 'SN': 1008,
 'SQ': 14,
 'TS': 1888,
 'UP': 230,
 'VC': 571}

## 만약 codesum에 들어간 code가 4개 이상인 것만 뽑아서 그중 빈도수가 가장 높은 code top3를 return 하도록 한다

In [43]:
def codesum_three(codesum):
    codesum = codesum.split(',')
    if len(codesum) > 3:
        tmp_code = []
        tmp_list = [counts_codesum[code] for code in codesum]
        tmp_list.sort()
        top3_code = tmp_list[-3:]
        for idx,val in counts_codesum.items():
            if val in top3_code:
                tmp_code.append(idx)
            else:
                continue
        return ",".join(tmp_code)
    return ",".join(codesum)

In [44]:
data.head()

,date,codesum
0,2012-01-01,"RA,FZ,FG,BR"
1,2012-01-01,RA
2,2012-01-01,MO
3,2012-01-01,MO
4,2012-01-01,MO


In [45]:
data_v1 = data.copy()

In [46]:
data_v1['codesum'] = data_v1['codesum'].apply(codesum_three)

In [47]:
data_v1.head()

,date,codesum
0,2012-01-01,"FG,RA,BR"
1,2012-01-01,RA
2,2012-01-01,MO
3,2012-01-01,MO
4,2012-01-01,MO


In [48]:
tmp_list = list(data_v1['codesum'])

In [49]:
counts_set = []
for i in tmp_list:
    i = ','.join(list(set(i.split(','))))
    counts_set.append(i)

In [50]:
counts_codesum_set = dict(Counter(counts_set))

# 유형 2-1. 100개 이상

- 전체 중 94.67%
- 카테고리 16개 + another code 1 = 17 개

In [51]:
code_name = []
sum_percentage = 0
for idx,val in counts_codesum_set.items():
    
    if val > 100:
        percentage = round((val / 20517) * 100,2)
        print(idx," "*(12 - len(idx)),val," "*(7 - len(str(val))),str(percentage)+"%")
        sum_percentage += percentage
        
        code_name.append(idx)
            
print("-------total_percentage---------")
print(str(sum_percentage) + "%")

SN            198      0.97%
RA,BR,TS      396      1.93%
RA,BR,HZ      235      1.15%
RA,TS         499      2.43%
SN,BR         252      1.23%
RA            1281     6.24%
TS            189      0.92%
RA,BR,DZ      130      0.63%
SN,RA,BR      119      0.58%
RA,BR         2046     9.97%
BR,RA,FG      166      0.81%
FG,BR,RA      195      0.95%
HZ            365      1.78%
BR,HZ         400      1.95%
MO            11051    53.86%
FG,BR         471      2.3%
BR            1431     6.97%
-------total_percentage---------
94.67%


In [52]:
(code_name)

['SN',
 'RA,BR,TS',
 'RA,BR,HZ',
 'RA,TS',
 'SN,BR',
 'RA',
 'TS',
 'RA,BR,DZ',
 'SN,RA,BR',
 'RA,BR',
 'BR,RA,FG',
 'FG,BR,RA',
 'HZ',
 'BR,HZ',
 'MO',
 'FG,BR',
 'BR']

### 16개 code 이외에는 'AN' 으로 분류

In [53]:
def last_classification(codesum):
    if ",".join(list(set(codesum.split(',')))) in code_name:
        return codesum
    else:
        return "AN"

In [54]:
data_v1['codesum'] = data_v1['codesum'].apply(last_classification)

In [55]:
data_v1.tail()

,date,codesum
20512,2014-10-31,MO
20513,2014-10-31,RA
20514,2014-10-31,MO
20515,2014-10-31,AN
20516,2014-10-31,MO


In [56]:
weather_v2 = weather.copy()

In [57]:
weather_v2['codesum'] = data_v1['codesum']

## 100개 이상인 코드 조합에 대해 나눈 것을 weather_v2.csv 로 저장

In [58]:
weather_v2.to_csv('../data/weather_v2.csv', sep = ',')

# 유형 2-2. 200개 이상
- 전체 중 89.81%
- 카테고리 12개 + another code 1 = 13 개

In [69]:
code_name = []
sum_percentage = 0
for idx,val in counts_codesum_set.items():
    
    if val > 200:
        percentage = round((val / 20517) * 100,2)
        print(idx," "*(12 - len(idx)),val," "*(7 - len(str(val))),str(percentage)+"%")
        sum_percentage += percentage
        
        code_name.append(idx)
            
print("-------total_percentage---------")
print(str(sum_percentage) + "%")

RA,BR,TS      396      1.93%
RA,BR,HZ      235      1.15%
RA,TS         499      2.43%
SN,BR         252      1.23%
RA            1281     6.24%
RA,BR         2046     9.97%
HZ            365      1.78%
BR,HZ         400      1.95%
MO            11051    53.86%
FG,BR         471      2.3%
BR            1431     6.97%
-------total_percentage---------
89.81%


In [60]:
data_v2 = data.copy()

In [61]:
data_v2['codesum'] = data_v2['codesum'].apply(codesum_three)
data_v2.head()

,date,codesum
0,2012-01-01,"FG,RA,BR"
1,2012-01-01,RA
2,2012-01-01,MO
3,2012-01-01,MO
4,2012-01-01,MO


In [62]:
code_name

['RA,BR,TS',
 'RA,BR,HZ',
 'RA,TS',
 'SN,BR',
 'RA',
 'RA,BR',
 'HZ',
 'BR,HZ',
 'MO',
 'FG,BR',
 'BR']

In [63]:
def last_classification(codesum):
    if ",".join(list(set(codesum.split(',')))) in code_name:
        return codesum
    else:
        return "AN"

In [64]:
data_v2['codesum'] = data_v2['codesum'].apply(last_classification)

In [65]:
data_v2.tail()

,date,codesum
20512,2014-10-31,MO
20513,2014-10-31,RA
20514,2014-10-31,MO
20515,2014-10-31,AN
20516,2014-10-31,MO


In [66]:
weather_v3 = weather.copy()

In [67]:
weather_v3['codesum'] = data_v2['codesum']

## 200개 이상인 코드 조합에 대해 나눈 것을 weather_v3.csv 로 저장

In [68]:
weather_v3.to_csv('../data/weather_v3.csv', sep = ',')